In [ ]:
import numpy as np
import scipy as sc

In [ ]:
# Testsystem aufsetzen
import control as con

# pt2 System
K = 1
d = 0.5
T = 10
delay = 0

a0 = 1
a1 = (2 * d * T) #16
a2 = (T**2) #100
b0 = K

# Polynom
tf_1 = con.matlab.tf(K, [a2, a1, a0])
#print tf_1

# Zustandsraum
ss_1a = con.matlab.tf2ss(tf_1)
#print ss_1a

# Füge Zeitversatz zu
d_num, d_den = con.pade(delay, 1)
tf_delay = con.tf(d_num, d_den)
ss_delay = con.series(tf_delay, tf_1)

#print con.matlab.tf2ss(ss_delay)

In [ ]:
import matplotlib.pyplot as plt
%pylab inline
d_yout, d_T = con.matlab.step(ss_delay)
yout, T = con.matlab.step(tf_1) # step without delay

plt.plot(d_T, d_yout, 'r-', label='poly_est')
plt.plot(np.add(d_T, delay), yout, 'g-', label='idealized') #delay in timeaxis!
plt.legend()

In [ ]:
# Sammle Systemteile (sinnlos in diesm Beispiel, aber der Vollständigkeit halber)
# original System Matrizen
A0 = ss_1a.A
b0 = ss_1a.B
C0 = ss_1a.C
d0 = ss_1a.D

# Platzierte pole
a_hat = np.matrix([[-0.2], [-0.2]]) # kA... einfach mal schnellere Pole machen

u_max = 0.1 # kA... Einfach mal begrenzen

In [ ]:
# Zustandslose Hilfsfunktionen
def _D(v, n):
    return np.diag([v**x for x in range(1, n+1)])

def _k(v, D, a_hat, a, n):
    D_inv = np.linalg.inv(D(v, n))
    #print np.subtract(np.dot(D_inv, a_hat), a) == D_inv.dot(a_hat) - a
    return D_inv.dot(a_hat) - a

def _R(v, D, R1):
    D_inv = np.linalg.inv(D(v, n))
    return np.dot(np.dot(D_inv, R1), D_inv)

def _e(v, k, D, a_hat, a, R, R1, u_max):
    return 1.0/u_max**2 * np.dot(np.dot(_k(v, D, a_hat, a, n).transpose(), np.linalg.inv(R(v, D, R1))), _k(v, D, a_hat, a, n))

# 1.te Ableitung e nach v
def _d_e(v, D, N, R1, n):
    D_inv = np.linalg.inv(D(v, n))
#    return 1.0/v*(D_inv*(N*R1 + R1*N)*D_inv)
    return 1.0/v * D_inv.dot(N.dot(R1) + R1.dot(N)).dot(D_inv)

# R1 muss symmetrisch sein!
def _R1_from_arr(r1, n):
    if len(r1) != ((n+1)*n)//2:
        raise ValueError("r1 has not right length! Has {} -> Needs {}".format(len(r1), ((n+1)*n)//2))

    R1 = np.zeros([n, n])
    r1_pos = 0
    for i in xrange(0, n):
        for j in xrange(i, n):
            R1[i,j] = r1[r1_pos]
            if i != j:
                R1[j,i] = r1[r1_pos]
            #print i, j, r1_pos
            #print r1[r1_pos]
            #print R1
            r1_pos += 1
    return R1

def initializeR1(n):
    return np.diag(np.random.rand(n)) # Immer positiv semidefinit für random  im Bereich von [0,1)

In [ ]:
## Hilfsfunktionen nur abhängig von Laufzeitvariablen
## Umwandlung des Systems in gewünschte Form

## Im scope definiter Variablen:
#
# A0, b0, C0, d0 -> Systemmatrizen
# a_hat -> gewünschte pole
# u_max -> +/- limit von u

# Transformation in Regelungsnormalform
ss, T = con.canonical_form(con.ss(A0, b0, C0, d0), form='reachable')

n = len(b0)
N = np.diag([p for p in xrange(-n, 0, 1)])

A = ss.A
a = (np.matrix(ss.A[0][:])).transpose()
b = np.matrix(ss.B).transpose()
C = np.matrix(ss.C)
d = np.matrix(ss.D) # == 0!

# D(v)
D = lambda v: _D(v, n)

# k(v)
k = lambda v: _k(v, _D, a_hat, a, n)

# R(v, R1)
R = lambda v, R1: _R(v, _D, R1)

# e(v, R1)
e = lambda v, R1: _e(v, k, _D, a_hat, a, _R, R1, u_max)

# g(v, x, R1)
g = lambda v, x, R1: e(v, R1) * x.transpose().dot(R1).dot(x) - 1.0

# d_e(v, R1)
d_e = lambda v, R1: _d_e(v, _D, N, R1, n)

In [ ]:
# Bedingungen

# Test auf positiv semi-definiteit (alle Eigenwerte >= 0)
def isPSD2(A, tol=1e-8):
    E,V = np.linalg.eigh(A)
    return np.all(E > -tol)

# Etwas schneller, ist positiv semi definit
def isPSD(A, tol=1e-8):
    w = np.linalg.eigvalsh(A)
    return np.all(w > -tol), w

def isNSD_sum(A, B, tol=1e-8):
    T = A.transpose().dot(B) + B.dot(A)
    w = np.linalg.eigvalsh(T)
    return np.all(w < tol), w

In [ ]:
#A = np.random.rand(3,3)
#%timeit isPSD(A)
#%timeit isPSD2(A)
print isPSD(initializeR1(3))

In [ ]:
import pprint

pp = pprint.PrettyPrinter(indent=4)
pretty = lambda x: pp.pprint(x)

print "A:"
pretty(A)
print "a:"
pretty(a)
print "b:"
pretty(b)
print "C:"
pretty(C)
print "d:"
pretty(d)

print "a_hat:"
pretty(a_hat)
print "n:", n
print "N:"
pretty(N)

In [ ]:
D1 = D(1)

k1 = k(1)
#print k_1 == k1

# Gebiet des Zustandsbereiches für x (hier Rechteckig)
X1 = [np.matrix([0, 0]).transpose(),
      np.matrix([0, 1]).transpose(),
      np.matrix([1, 1]).transpose(),
      np.matrix([1, 0]).transpose()]

A1_hat = A - b.dot(k(1))
#print A1_hat

#http://deap.readthedocs.io/en/master/tutorials/advanced/constraints.html

In [ ]:
def eval_WSVR(individual):
    #print "Entering"
    pain = [0, 0, 0, 0, 0, 0]
    R1 = _R1_from_arr(individual, n)
    try:
        psd, w = isPSD(R1)
        if not psd:
            pain[0] = np.sum(np.abs([wp for wp in w if wp < 0]))

        nsd, w = isNSD_sum(A1_hat, N)
        if not nsd:
            pain[1] = np.sum(np.abs([wp for wp in w if wp > 0]))

        nsd, w = isNSD_sum(N, R1)
        if not nsd:
            pain[2] = np.sum(np.abs([wp for wp in w if wp > 0]))

        v_arr = np.arange(0.0001, 1, 0.0001)
        de_max = np.max([d_e(v, R1) for v in v_arr])
        if not de_max <= 0:
            pain[3] = de_max

        G1 = [g(1, x, R1) for x in X1]
        if not np.max(G1) < 0:
            pain[4] = np.max(G1)
    except:
        pain[5] = numpy.float64(10000)
    finally:
        #print np.sum(pain), "\n", pain, "\n->", individual
        #return [np.sum(pain)]
        return pain

# Initialize deap population
import random
from deap import algorithms, base, creator, tools

creator.create("FitnessMin", base.Fitness, weights=(-1.0,
                                                    -1.0,
                                                    -1.0,
                                                    -1.0,
                                                    -1.0,
                                                    -10.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)


toolbox = base.Toolbox()
toolbox.register("attr_float", random.random) # float value between [0,1)
toolbox.register("individual", tools.initRepeat,
                               creator.Individual,
                               toolbox.attr_float,
                               n=3)

toolbox.register("population", tools.initRepeat,
                               list, 
                               toolbox.individual)

toolbox.register("evaluate", eval_WSVR)

toolbox.register("mutate", tools.mutGaussian, mu=0.0,
                                              sigma=1.0,
                                              indpb=0.2)
toolbox.register("mate", tools.cxTwoPoint)

toolbox.register("select", tools.selTournament, tournsize=3)

In [ ]:
# Creates and starts the algorithm
pop = toolbox.population(n=10)

winpop, logbook = algorithms.eaMuPlusLambda(pop, 
                              toolbox, 
                              mu = 5, 
                              lambda_ = 10, 
                              cxpb = 0.5, 
                              mutpb = 0.5, 
                              ngen = 10,
                              #[, stats, halloffame, verbose]
                              verbose = True
                             )

In [ ]:
best = tools.selBest(winpop, k=1)
print best
print eval_WSVR(best[0])